In [5]:
# import statements
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.tree import export_text
from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# read in preprocessed data
data = pd.read_csv("../data/preprocessed/preprocessed_crime_data.csv")

In [11]:
data.head()

,state,county,community,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop,IncomeInequalityRatio,PoliceToIncomeRatio,EconomicHardshipIndex
0,-1.420429,0.310794,0.037312,-1.547791,4.858444,-0.971634,-0.581079,0.381995,0.112572,0.599924,...,-5.045534,-3.813790,9.604056,1.099440,3.734827,-4.133042,0.141179,0.048525,-0.852938,-0.354893
1,1.437993,0.310794,0.037312,-1.547791,-0.821592,-2.404197,-0.017970,-0.627322,2.876804,-0.146559,...,0.157901,0.164086,-0.077150,0.133873,-0.157562,0.122692,3.028246,-1.753752,-0.129869,-0.000679
2,-0.404101,0.310794,0.037312,-1.547791,-0.821592,-0.213218,2.065530,-1.762805,0.531395,-0.370505,...,0.157901,0.164086,-0.077150,0.133873,-0.157562,0.122692,1.553999,-0.888134,-0.081588,0.802208
3,0.739268,1.548650,-2.364854,-1.547791,-0.522643,0.882271,-0.581079,0.697407,-0.138722,-0.295856,...,0.157901,0.164086,-0.077150,0.133873,-0.157562,0.122692,-0.903080,0.161003,-0.680276,-1.205009
4,0.866309,-1.462734,-0.243593,-1.547791,-0.522643,-0.466024,-0.693700,0.886654,-0.390016,-0.519801,...,0.157901,0.164086,-0.077150,0.133873,-0.157562,0.122692,-0.903080,1.120199,-0.509222,-0.543808


To predict `ViolentCrimesPerPop` we begin with a Linear regression model for initial exploration, but move to Ridge and Lasso to address overfitting by implementing regularization techniques. 

### Linear Regression 

In [17]:
# Features variable (all but target)
X = data.drop('ViolentCrimesPerPop', axis = 1)

# Target variable
y = data['ViolentCrimesPerPop']

# Split the data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [18]:
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Only transform the test data using the already fitted scaler
X_test_scaled = scaler.transform(X_test)

In [19]:
# Initialize the linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train_scaled, y_train)

LinearRegression()

In [20]:
# Predict the target variable on the test set
y_pred = model.predict(X_test_scaled)

In [21]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate R-squared value
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Squared Error: 1.6582127180139292e+24
R-squared: -1.3946518024271707e+24


The results indicate a very large MSE, suggesting that the model is performing very poorly. The negative R squared indicates that the model is misfitting the data severely due to issues such as multicollinearity and overfitting. We now explore Ridge and Lasso regression techniques to address the issue of overfitting and optimize our models. 

### Lasso 

In [24]:
from sklearn.linear_model import Lasso

# Initialize the Lasso model
lasso = Lasso(alpha=0.1) 

# Train the Lasso model on the scaled training data
lasso.fit(X_train_scaled, y_train)

# Predict the target variable on the test set
y_pred = lasso.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Squared Error: 0.5506070664852809
R-squared: 0.5369079434980045


#### Tuning Lasso with Cross-Validation

In [25]:
from sklearn.model_selection import GridSearchCV

# Define a range of alpha values to test
param_grid = {'alpha': np.logspace(-4, 4, 50)}

# Perform GridSearchCV to find the best alpha value
grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Print the best alpha and corresponding performance
print("Best alpha:", grid_search.best_params_['alpha'])
best_lasso = grid_search.best_estimator_

# Predict with the best model
y_pred = best_lasso.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


C:\Users\Samantha\anaconda_work\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.407e+01, tolerance: 5.250e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Samantha\anaconda_work\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.447e+01, tolerance: 4.885e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Samantha\anaconda_work\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap

Best alpha: 0.05963623316594643
Mean Squared Error: 0.5351326218855517
R-squared: 0.5499228370747589


With initial alpha=0.1, the MSE is 0.5506 suggests that the model's predictions are somewhat accurate but there is still some error. Further, the R squared indicates that the model explains about 53.7% of the variability in the target variable. While not perfect, it is a reasonable performance for noisy data. We tune the alpha value using grid search for a better outcome. After tuning alpha to = 0.0596, our MSE improves slightly to 0.5351 while our R squared increases slightly to 0.5499, suggesting that the model is better at explaining variance in the target variable with the new alpha value. 

The alpha parameter controls regularization strength and the tuning reveals that a slightly stronger regulartization with a smaller alpha helps reduce overfitting. 

### Ridge 

In [27]:
from sklearn.linear_model import Ridge

# Initialize the Ridge model
ridge = Ridge(alpha=1.0) 

# Train the Ridge model on the scaled training data
ridge.fit(X_train_scaled, y_train)

# Predict the target variable on the test set
y_pred = ridge.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Squared Error: 0.7228496067821413
R-squared: 0.3920421089340502


#### Tuning Ridge with Cross-Validation

In [28]:
from sklearn.model_selection import GridSearchCV

# Define a range of alpha values to test
param_grid = {'alpha': np.logspace(-4, 4, 50)}

# Perform GridSearchCV to find the best alpha value
grid_search = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Print the best alpha and corresponding performance
print("Best alpha:", grid_search.best_params_['alpha'])
best_ridge = grid_search.best_estimator_

# Predict with the best model
y_pred = best_ridge.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Best alpha: 494.1713361323828
Mean Squared Error: 0.5294332754256078
R-squared: 0.5547163136454474


When alpha = 0.1, the MSE suggests that the predictions are relatively off from actual values and that the model is not regularizing enough and is overfitting values. The R squared is also low indicating that this model is only moderatly effective at capturing relationships in the data.

After tuning with the best alpha, we find that alpha = 494.171 is the optimal value, meaning regularization is very strong, forcing the model to shrink coefficients more significantly to prevent overfitting by penalizing large coefficients. The MSE is smaller than that of the previous Ridge model as well as the optimized Lasso model above. The R squared value also significantly improved from the first Ridge model. 

The Ridge model using the best alpha yields the best results overall, with a low MSE and high R-squared compared to the linear regression and lasso models. 